## Input result location for each model

In [ ]:
import os

PATH_LIST = [
    'Please fill the path of each model, there should be 5 rounds in each folder',
]

BASE_PATH_LIST = []
for i in PATH_LIST:
    tempList = []
    exps = os.listdir(i)
    exps.remove('meta.yaml')
    for j in exps:
        tempList.append(i + j + "/artifacts/")
    BASE_PATH_LIST.append(tempList)
BASE_PATH_LIST

## Extract prediction and ground-truth return rate 

In [6]:
import qlib
from qlib.contrib.data.handler import Alpha360
from qlib.data.dataset.processor import RobustZScoreNorm, Fillna, DropnaLabel, CSRankNorm
from qlib.data.dataset import DatasetH, DataHandlerLP
import pandas as pd

provider_uri = "Please fill the location of Qlib data"

qlib.init(provider_uri=provider_uri)
EXP_VERSION = "M1"

'''Model Config'''
START_TIME = '20180101'
END_TIME = '20221231'
FIT_START = '20180101'
FIT_END = '20221231'
VALID_START = '20180101'
VALID_END = '20221231'
TEST_START = '20180101'
TEST_END = '20221231'

dh = Alpha360(instruments='csi300jq', start_time=START_TIME, end_time=END_TIME,
              fit_start_time=FIT_START,
              fit_end_time=FIT_END,
              infer_processors=[
                  RobustZScoreNorm(fit_start_time=FIT_START, fit_end_time=FIT_END, fields_group="feature",
                                   clip_outlier="true"),
                  Fillna(fields_group="feature"),
              ],
              learn_processors=[
                  DropnaLabel(),
                  CSRankNorm(fields_group="label")
              ],
              label=(["Ref($close, -2) / Ref($close, -1) - 1"], ["LABEL"]))
ds = DatasetH(handler=dh, segments={"train": (FIT_START, FIT_END), "valid": (VALID_START, VALID_END), "test": (TEST_START, TEST_END)})
df_test = ds.prepare("test", col_set=["label"], data_key=DataHandlerLP.DK_L)
df_ret = ds.prepare("test", col_set=["label"], data_key=DataHandlerLP.DK_R)

for i in BASE_PATH_LIST:
    for j in i:
        df_ret.to_pickle(j+"ret.pkl")

for i in BASE_PATH_LIST:
    for BASE_PATH in i:
        df_score = pd.read_pickle(BASE_PATH+"pred.pkl")
        df_final = pd.concat([df_test,df_score['score']],axis=1)
        df_final.columns = ['label','score']
        df_final.to_pickle(BASE_PATH+"/pred.pkl")

print("Done.")

[14258:MainThread](2024-02-07 14:41:09,280) INFO - qlib.Initialization - [config.py:416] - default_conf: client.
[14258:MainThread](2024-02-07 14:41:09,284) INFO - qlib.Initialization - [__init__.py:74] - qlib successfully initialized based on client settings.
[14258:MainThread](2024-02-07 14:41:09,285) INFO - qlib.Initialization - [__init__.py:76] - data_path={'__DEFAULT_FREQ': PosixPath('/data/zuyi/qlib_data/kd/jqdata_new')}
[14258:MainThread](2024-02-07 14:41:26,835) INFO - qlib.timer - [log.py:127] - Time cost: 17.547s | Loading data Done
/home/zuyi/anaconda3/envs/fusion/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1095: RuntimeWarning: All-NaN slice encountered
  result = np.apply_along_axis(_nanmedian1d, axis, a, overwrite_input)
[14258:MainThread](2024-02-07 14:41:57,324) INFO - qlib.timer - [log.py:127] - Time cost: 30.072s | RobustZScoreNorm Done
[14258:MainThread](2024-02-07 14:41:57,853) INFO - qlib.timer - [log.py:127] - Time cost: 0.527s | Fillna Done
[14258:MainT

Done.


## Metric Definition

In [7]:
import numpy as np
import pandas as pd
from tqdm.auto import tqdm

def backtest(fname, **kwargs):
    pred = pd.read_pickle(fname) # test period
    ic = pred.groupby(level='datetime').apply(lambda x: x.label.corr(x.score))
    rank_ic = pred.groupby(level='datetime').apply(lambda x: x.label.corr(x.score, method='spearman'))

    return {
        'IC': ic.mean(),
        'ICIR': ic.mean()/ic.std(),
        'RankIC': rank_ic.mean(),
        'RankICIR': rank_ic.mean()/rank_ic.std()
    }

def fmt(x, p=3, scale=1, std=False):
    _fmt = '{:.%df}'%p
    string = _fmt.format((x.mean() if not isinstance(x, (float, np.floating)) else x) * scale)
    if std and len(x) > 1:
        string += ' ('+_fmt.format(x.std()*scale)+')'
    return string

def backtest_multi(files, **kwargs):
    res = []
    for fname in files:
        metric= backtest(fname, **kwargs)
        res.append(metric)
    res = pd.DataFrame(res)
    return {
        'IC': fmt(res['IC'], std=True),
        'ICIR': fmt(res['ICIR'], std=True),
        'RankIC': fmt(res['RankIC'], std=True),
        'RankICIR': fmt(res['RankICIR'], std=True),
    },''

In [8]:
BASE_PATH_BT = []

for i in BASE_PATH_LIST:
    tempList = []
    for BASE_PATH in i:
        tempList.append(BASE_PATH+"pred.pkl")
    BASE_PATH_BT.append(tempList)

exps = {
    'Finformer': BASE_PATH_BT[0],
    'ALSTM': BASE_PATH_BT[1],
    'GAT': BASE_PATH_BT[2],
    'HIST': BASE_PATH_BT[3],
    'MLP': BASE_PATH_BT[4],
    'TCN': BASE_PATH_BT[5],
    'TRA+ALSTM': BASE_PATH_BT[6],
    'Transformer': BASE_PATH_BT[7],
    'CrossFormer': BASE_PATH_BT[8],
    'SFM': BASE_PATH_BT[9],
    'DTML': BASE_PATH_BT[10],
    'THGNN': BASE_PATH_BT[11],
}

In [9]:
res = {
    name: backtest_multi(exps[name])
    for name in tqdm(exps)
}
report = pd.DataFrame({
    k: v[0]
    for k, v in res.items()
}).T

  0%|          | 0/12 [00:00<?, ?it/s]

In [10]:
report

,IC,ICIR,RankIC,RankICIR
Finformer,0.052 (0.002),0.359 (0.012),0.050 (0.002),0.345 (0.013)
ALSTM,0.045 (0.001),0.339 (0.010),0.043 (0.001),0.330 (0.011)
GAT,0.038 (0.003),0.250 (0.036),0.036 (0.004),0.234 (0.042)
HIST,0.049 (0.004),0.293 (0.029),0.047 (0.004),0.279 (0.028)
MLP,0.024 (0.003),0.186 (0.027),0.022 (0.003),0.177 (0.028)
TCN,0.039 (0.001),0.289 (0.020),0.035 (0.001),0.265 (0.018)
TRA+ALSTM,0.043 (0.003),0.338 (0.030),0.042 (0.003),0.332 (0.028)
Transformer,0.023 (0.002),0.145 (0.015),0.024 (0.002),0.154 (0.019)
CrossFormer,0.040 (0.002),0.306 (0.026),0.039 (0.002),0.301 (0.027)
SFM,0.037 (0.003),0.285 (0.023),0.033 (0.004),0.251 (0.030)
